<a href="https://colab.research.google.com/github/YaninaK/anomaly-detection/blob/b1/notebooks/02_Anomalies_1_2_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обнаружение аномалий в начислениях за тепловую энергию


## 2. Аномалии 1, 2 и 4.



### Примеры аномалий

Виды аномалий по показаниям приборов учёта тепловой энергии, которые необходимо выявлять (кроме объектов с видом энергопотребления ГВС (централ):

1. нулевые значения показаний за тепловую энергию в отопительный период (октябрь-апрель);

2. равные значения показаний в течение нескольких расчетных периодов;

3. снижение/рост показаний в отдельные месяцы по сравнению с показаниями за предыдущие периоды по данному объекту (с учётом фактической температуры наружного воздуха и количества отопительных дней в месяце);

4. аномально низкое/высокое (отклонение более 25%) потребление объекта в конкретном месяце по сравнению с аналогичными объектами (только для типов объекта «Многоквартирный дом») по критериям:
  - год постройки (по группам до 1958 г., 1959-1989 гг., 1990-2000 гг., 2001-2010 гг., 2011-2024 гг.),
  - этажность (по группам 1-2 этажа, 3-4 этажа, 5-9 этажей,10-12 этажей, 13 и более этажей),
  - площадь (±10%),
  - наличие ГВС ИТП (горячей воды, учитываемой тем же прибором).



In [1]:
initiate = True
if initiate:
  !git init -q
  !git clone -b b1  https://github.com/YaninaK/anomaly-detection.git -q

  from google.colab import drive
  drive.mount('/content/drive')

  !unzip -u -q /content/drive/MyDrive/ML_projects/08_anomaly_detection/data/01_raw/task#3.zip -d /content/anomaly-detection/data/01_raw

%cd /content/anomaly-detection/notebooks

Mounted at /content/drive
/content/anomaly-detection/notebooks


In [2]:
import os
import sys

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "..", "src", "anomaly_detection"))

In [3]:
import datetime

import numpy as np
import pandas as pd

from data.make_dataset import load_data
from data.preprocess import Preprocess
from data.find_missing import find_missing_records, get_uninvoiced_buildings
from data.duplicated import get_equal_values
from features.grouping import Grouping
from features.period_outliers import get_outlers

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
PATH = "/content/anomaly-detection/"

## 1. Чтение данных

In [6]:
folder_path = '../data/01_raw/'

In [7]:
regenerate = True
data, temperature, buildings = load_data(folder_path, regenerate, path=PATH)

100%|██████████| 24/24 [00:09<00:00,  2.47it/s]


## 2. Подготовка данных

In [8]:
preprocess = Preprocess()
data, buildings = preprocess.fit_transform(data, buildings)

## 3. Аномалии



### 3.1 Нулевые значения показаний за тепловую энергию в отопительный период (октябрь-апрель)

#### 3.1.1 Пропуски в данных по текущему потреблению

In [9]:
save = True
missing_records_addr, missing_records_df = find_missing_records(data, save, path=PATH)
n_missing_records = missing_records_df.iloc[:, 2:].isnull().sum().sum()

print(f"missing_records_df.shape = {missing_records_df.shape}")
print(f"Всего нулевых значений: {n_missing_records}\n")
missing_records_df.sample(2)

missing_records_df.shape = (776, 16)
Всего нулевых значений: 3519



Период потребления,Адрес объекта,Тип объекта,2021-10-01 00:00:00,2021-11-01 00:00:00,2021-12-01 00:00:00,2022-01-01 00:00:00,2022-02-01 00:00:00,2022-03-01 00:00:00,2022-04-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00
382,"г Уфа, ул. Летчиков, д.2 корп.5",Многоквартирный дом,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.207,130.725,246.179,221.572,167.815,115.501,43.676
704,"г Уфа, ул. Цветочная, д.3",Другое строение,52.899,106.366,126.085,168.221,139.425,120.624,89.411,NaN,91.757,165.770,176.005,139.602,103.322,72.710


In [10]:
for k in missing_records_addr.keys():
  print(f"{k.strftime('%Y-%m')}: {len(missing_records_addr[k])}")

2021-10: 418
2021-11: 288
2021-12: 270
2022-01: 285
2022-02: 276
2022-03: 281
2022-04: 304
2022-10: 345
2022-11: 182
2022-12: 163
2023-01: 176
2023-02: 162
2023-03: 160
2023-04: 209


#### 3.1.2 Отсутствуют данные по текущему потреблению

In [11]:
save = True
uninvoiced_buildings = get_uninvoiced_buildings(
    data, buildings, save, path=PATH
)
print(f"uninvoiced_buildings.shape = {uninvoiced_buildings.shape}\n")
uninvoiced_buildings.iloc[:, :-1].sample(2)

uninvoiced_buildings.shape = (1650, 6)



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
4613,"г Уфа, ул. Российская, д.43",Другое строение,10,1980-01-01,0.01
3962,"г Уфа, ул. Пекинская, д.20 корп.а",Многоквартирный дом,2,1958-01-01,298.90


In [12]:
pd.concat(
    [
        uninvoiced_buildings["Тип Объекта"].value_counts(),
        uninvoiced_buildings["Тип Объекта"].value_counts(normalize=True)
    ], axis=1
).head(10)

,count,proportion
Тип Объекта,,
Многоквартирный дом,752,0.455758
Другое строение,652,0.395152
Частный дом,138,0.083636
"Административные здания, конторы",45,0.027273
"Учебное заведение, комбинат, центр",13,0.007879
Школы и ВУЗ,13,0.007879
Гаражи,6,0.003636
"Жилое здание (Гостиница, Общежитие)",5,0.003030
Производственный объект,4,0.002424


In [13]:
cond = uninvoiced_buildings["Тип Объекта"] == "Многоквартирный дом"
s = uninvoiced_buildings[cond]['Общая площадь объекта'].sum() / 1e06

print(f"У {cond.sum()} объектов типа Многоквартирный дом общей площадью{s: 0.1f} млн. нет данных о выствленных счетах.")

У 752 объектов типа Многоквартирный дом общей площадью 1.1 млн. нет данных о выствленных счетах.


* У 1650 объектов нет данных об учете потребления теплоэнергии. Из них 752 объекта (46%) общей площадью 1.1 млн. приходится на тип ```Многоквартирный дом```, около 40% - на тип ```Другое строение```, около 8.3% - на тип ```Частный дом```.


#### 3.1.3 Неуникальные адреса объектов.

In [14]:
df = buildings[
    buildings.duplicated(subset=["Адрес объекта", "Тип Объекта"], keep=False)
]
print(f"Всего неуникальных объектов: {df.shape[0]}\n")
df.head(6)

Всего неуникальных объектов: 145



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2
122,"г Уфа, б-р. Тухвата Янаби, д.34",Другое строение,1,NaT,0.01,"г Уфа, б-р. Тухвата Янаби, д.34"
123,"г Уфа, б-р. Тухвата Янаби, д.34",Другое строение,2,NaT,0.01,"г Уфа, б-р. Тухвата Янаби, д.34"
412,"г Уфа, пр-кт Октября, д.153",Другое строение,0,NaT,0.01,"г Уфа, пр-кт Октября, д.153"
413,"г Уфа, пр-кт Октября, д.153",Другое строение,3,NaT,0.01,"г Уфа, пр-кт Октября, д.153"
487,"г Уфа, пр-кт Октября, д.31",Другое строение,0,1970-01-01,0.01,"г Уфа, пр-кт Октября, д.31"
488,"г Уфа, пр-кт Октября, д.31",Другое строение,5,1970-01-01,0.01,"г Уфа, пр-кт Октября, д.31"


In [15]:
pd.concat(
    [
      df["Тип Объекта"].value_counts(),
      df["Тип Объекта"].value_counts(True),
      df.groupby(["Тип Объекта"])["Адрес объекта"].agg(lambda x: x.nunique())
    ], axis=1
)

,count,proportion,Адрес объекта
Тип Объекта,,,
Другое строение,116,0.800000,51
Многоквартирный дом,14,0.096552,7
Частный дом,12,0.082759,4
"Учебное заведение, комбинат, центр",3,0.020690,1


1. Типы объектов ```Другое строение``` и ```Частный дом``` оставлены без корректировок:
  * Уникальный адрес объекта нужен для привязки площади объекта и даты постройки к данным учета потребления теплоэнергии. У типа ```Другое строение``` - в большинстве случаев эти данные отстутствуют, соответственно этот блок информации не имеет смысла корректировать.

  * Информации об неуникальных объектах типа ```Частный дом``` нет в данных о потреблении теплоэнергии - этот блок также не имеет смысла корректировать.

2. Блок ```Многоквартирный дом``` скорректирован:
  * Адреса многоквартирных домов сделаны уникальными в признаке ```Адрес объекта 2``` через сопоставление с данными об учете теплоэнергии.
  * На трех объектах ниже счета не выставляются, для них в признаке ```Адрес объекта 2``` к адресу добавлено слово ```extra``` :
    * ```г Уфа, ул. Вологодская, д.20```,
    * ```г Уфа, ул. Интернациональная, д.113```,
    * ```г Уфа, ул. Нежинская, д.6```.
3. Блок ```Учебное заведение, комбинат, центр``` скорректирован:
  * Адреса сделаны уникальными в признаке ```Адрес объекта 2``` через сопоставление с данными об учете теплоэнергии и проставление ``№ ОДПУ`` в адрес.

In [16]:
df = buildings[
    buildings.duplicated(subset=["Адрес объекта 2", "Тип Объекта"], keep=False)
]
pd.concat(
    [
      df["Тип Объекта"].value_counts(),
      df["Тип Объекта"].value_counts(True),
      df.groupby(["Тип Объекта"])["Адрес объекта 2"].agg(lambda x: x.nunique())
    ], axis=1
)

,count,proportion,Адрес объекта 2
Тип Объекта,,,
Другое строение,115,0.905512,51
Частный дом,12,0.094488,4


* Если использовать ```Адрес объекта 2```, неуникальные адреса в типах  ```Многоквартирный дом``` и ```Учебное заведение, комбинат, центр``` появляться не будут.

## 3.2 Равные значения показаний в течение нескольких расчетных периодов

In [17]:
save = True
equal_values = get_equal_values(data, save, path=PATH)

print(f"Число записей с равными значениями показаний: {equal_values.shape[0]}")
print(f"Число адресов с равными значениями показаний: {equal_values['Адрес объекта'].nunique()}\n")
equal_values.head(10)

Число записей с равными значениями показаний: 256
Число адресов с равными значениями показаний: 101



,index,Подразделение,№ ОДПУ,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания,"Текущее потребление, Гкал",Период потребления
36031,3465.0,Уфа,787,ГВС-ИТП,г Уфа,Другое строение,2022-01-01,188.0,2021-12-01
50282,693.0,Уфа,787,ГВС-ИТП,г Уфа,Другое строение,2022-07-01,21.0,2022-06-01
48479,NaN,Уфа,787,ГВС-ИТП,г Уфа,Другое строение,2022-09-01,12.0,2022-08-01
59404,NaN,Уфа,787,ГВС-ИТП,г Уфа,Другое строение,2022-10-01,12.0,2022-09-01
0,NaN,Уфа,787,ГВС-ИТП,г Уфа,Другое строение,2023-02-01,188.0,2023-01-01
19672,NaN,Уфа,787,ГВС-ИТП,г Уфа,Другое строение,2023-07-01,21.0,2023-06-01
58934,3904.0,Уфа,27834,ГВС-ИТП,"г Уфа, б-р. Ибрагимова, д.61",Другое строение,2022-04-01,70.0,2022-03-01
42679,47.0,Уфа,27834,ГВС-ИТП,"г Уфа, б-р. Ибрагимова, д.61",Другое строение,2022-12-01,70.0,2022-11-01
47078,27.0,Уфа,27834,ГВС-ИТП,"г Уфа, б-р. Ибрагимова, д.61",Другое строение,2023-06-01,2.0,2023-05-01
19697,26.0,Уфа,27834,ГВС-ИТП,"г Уфа, б-р. Ибрагимова, д.61",Другое строение,2023-07-01,2.0,2023-06-01


## 3.3. Аномально низкое/высокое (отклонение более 25%) потребление объекта в конкретном месяце по сравнению с аналогичными объектами

(только для типов объекта «Многоквартирный дом») по критериям:
  - год постройки (по группам до 1958 г., 1959-1989 гг., 1990-2000 гг., 2001-2010 гг., 2011-2024 гг.),
  - этажность (по группам 1-2 этажа, 3-4 этажа, 5-9 этажей,10-12 этажей, 13 и более этажей),
  - площадь (±10%),
  - наличие ГВС ИТП (горячей воды, учитываемой тем же прибором).

In [18]:
grouping = Grouping()
df = grouping.fit_transform(data, buildings)

print(f"df.shape = {df.shape}\n")
df.sample(2)

df.shape = (3067, 35)



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2,Группа этажность объекта,Улица,Дата постройки 2,Группа год постройки,...,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00
1324,"г Уфа, ул. Карла Маркса, д.48 корп.1",Многоквартирный дом,9.0,2002-01-01,8280.2,"г Уфа, ул. Карла Маркса, д.48 корп.1",5-9 этажей,ул. Карла Маркса,2002-01-01,2001-2010 гг.,...,18.678,79.949,128.519,166.845,186.765,158.121,129.082,100.209,23.018,16.903
410,"г Уфа, пр-кт Октября, д.71 корп.2",Многоквартирный дом,20.0,1989-01-01,8337.4,"г Уфа, пр-кт Октября, д.71 корп.2",13 и более этажей,пр-кт Октября,1989-01-01,1959-1989 гг.,...,NaN,137.260,194.967,278.210,288.100,251.340,197.091,119.675,NaN,NaN


Удалены объекты с неуказанной общей площадью:

1. Нет информации ```Этажность объекта```, ```Дата постройки```, ```Общая площадь объекта``` по многоквартирному дому по адресу ```г Уфа, ул. Уфимское шоссе, д.4```.

3. По адресу ```г Уфа, ул. Кирова, д.95``` значатся 2 многоквартирных дома: 5 этажей и 1 этаж. По одноэтажному дому (```Подобъект №46590```) нет информации по общей площади объекта.

In [19]:
save=True

n_periods = 24
threshold = 0.25

under_medians, over_medians, underconsumption, overconsumption = (
    get_outlers(df, threshold=threshold, n_periods=n_periods,  save=save, path=PATH)
)
print(f"Aномально низкое/высокое (отклонение более {threshold:.0%}) потребление:")
for period in under_medians:
  print(f"{period: %Y-%m}:  {len(under_medians[period])}\t{len(over_medians[period])}")

Aномально низкое/высокое (отклонение более 25%) потребление:
 2021-07:  148	197
 2021-08:  175	186
 2021-09:  146	182
 2021-10:  78	123
 2021-11:  65	87
 2021-12:  71	71
 2022-01:  70	67
 2022-02:  71	67
 2022-03:  79	80
 2022-04:  92	112
 2022-05:  146	191
 2022-06:  130	186
 2022-07:  148	198
 2022-08:  170	211
 2022-09:  131	166
 2022-10:  98	136
 2022-11:  68	77
 2022-12:  73	77
 2023-01:  71	70
 2023-02:  65	74
 2023-03:  74	74
 2023-04:  113	118
 2023-05:  141	201
 2023-06:  166	216


In [20]:
print(f"underconsumption.shape = {underconsumption.shape}\n")
underconsumption.sample(5)

underconsumption.shape = (509, 35)



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2,Группа этажность объекта,Улица,Дата постройки 2,Группа год постройки,...,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00
1864,"г Уфа, ул. Менделеева, д.162",Многоквартирный дом,6.0,1991-01-01,2826.93,"г Уфа, ул. Менделеева, д.162",5-9 этажей,ул. Менделеева,1991-01-01,1990-2000 гг.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2361,"г Уфа, ул. Рихарда Зорге, д.34 корп.2",Многоквартирный дом,5.0,1965-01-01,3532.00,"г Уфа, ул. Рихарда Зорге, д.34 корп.2",5-9 этажей,ул. Рихарда Зорге,1965-01-01,1959-1989 гг.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.080
1318,"г Уфа, ул. Карла Маркса, д.36",Многоквартирный дом,5.0,1959-01-01,3634.80,"г Уфа, ул. Карла Маркса, д.36",5-9 этажей,ул. Карла Маркса,1959-01-01,до 1958 г,...,8.162,40.394,NaN,NaN,NaN,NaN,NaN,48.842,NaN,NaN
29,"г Уфа, б-р. Ибрагимова, д.44",Многоквартирный дом,10.0,1997-01-01,8515.90,"г Уфа, б-р. Ибрагимова, д.44",10-12 этажей,б-р. Ибрагимова,1997-01-01,1990-2000 гг.,...,16.344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.046,16.921
1360,"г Уфа, ул. Кирова, д.31",Многоквартирный дом,10.0,2000-01-01,9767.70,"г Уфа, ул. Кирова, д.31",10-12 этажей,ул. Кирова,2000-01-01,1990-2000 гг.,...,19.065,NaN,NaN,NaN,NaN,NaN,145.753,62.468,20.572,18.790


In [21]:
print(f"overconsumption.shape = {overconsumption.shape}\n")
overconsumption.sample(5)

overconsumption.shape = (558, 35)



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2,Группа этажность объекта,Улица,Дата постройки 2,Группа год постройки,...,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00
283,"г Уфа, пр-кт Октября, д.166 корп.1",Многоквартирный дом,5.0,1980-01-01,1829.6,"г Уфа, пр-кт Октября, д.166 корп.1",5-9 этажей,пр-кт Октября,1980-01-01,1959-1989 гг.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.802,8.871
221,"г Уфа, пр-кт Октября, д.127 корп.1",Многоквартирный дом,5.0,1962-01-01,2735.7,"г Уфа, пр-кт Октября, д.127 корп.1",5-9 этажей,пр-кт Октября,1962-01-01,1959-1989 гг.,...,NaN,56.473,NaN,NaN,NaN,NaN,NaN,73.403,NaN,NaN
391,"г Уфа, пр-кт Октября, д.64 корп.1",Многоквартирный дом,5.0,1964-01-01,3471.2,"г Уфа, пр-кт Октября, д.64 корп.1",5-9 этажей,пр-кт Октября,1964-01-01,1959-1989 гг.,...,20.544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.225
401,"г Уфа, пр-кт Октября, д.66 корп.3",Многоквартирный дом,5.0,1971-01-01,1799.6,"г Уфа, пр-кт Октября, д.66 корп.3",5-9 этажей,пр-кт Октября,1971-01-01,1959-1989 гг.,...,11.172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.883,12.072
1929,"г Уфа, ул. Мира, д.49",Многоквартирный дом,19.0,2020-01-01,8818.5,"г Уфа, ул. Мира, д.49",13 и более этажей,ул. Мира,2020-01-01,2011-2024 гг.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
